## Basic Usage

# Generating Community Parameters

The first step of any simulation is to generate a set of parameters for a given microbial community. In `micrm.py`, we can define some parameters, and generate uptake rate and leakage rate by calling functions in `param.py`. `np.random.seed` makes results repeatable. In this example we will consider a simple unstructured community where uptake and leakage values are randomly drawn from a dirchlet distribution and all other parameters are shown as in code:

In [ ]:
import numpy as np
import param

# This is a basic code for simulating MiCRM

np.random.seed(42)

# parameters
N = 10  # consumer number
M = 5  # resource number
λ = 0.1  # total leakage rate

N_modules = 2 # module number of consumer to resource
s_ratio = 10.0 

u = param.modular_uptake(N, M, N_modules, s_ratio)  # uptake matrix

lambda_alpha = np.full(M, λ)  # total leakage rate for each resource

m = np.full(N, 0.2)  # mortality rate of N consumers

rho = np.full(M, 0.5)  # input of M resources

omega = np.full(M, 0.5)  # decay rate of M resources

l = param.generate_l_tensor(N, M, N_modules, s_ratio, λ) # a tensor for all consumers' leakage matrics

# Defining System Dynamics

Once we have a set of parameters the next step is to define the dynamics of the community we want to simulate. `micrm.py` allows the user to specify the dynamics of the system they wish to simulate by passing custom functions to the ... function. 

This process is explained in more detail XXX but briefly the user can specify dynamics depending on the parameters created in the previous step as well as additional state variables to represent external drivers such as temperature. In this example we will use the default dynamics as described above and so we can just use the ... out of the box.



In [ ]:
def dCdt_Rdt(t, y):
    C = y[:N]
    R = y[N:]
    dCdt = np.zeros(N)
    dRdt = np.zeros(M)
    
    for i in range(N):
        dCdt[i] = sum(C[i] * R[alpha] * u[i, alpha] * (1 - lambda_alpha[alpha]) for alpha in range(M)) - C[i] * m[i]
    
    for alpha in range(M):
        dRdt[alpha] = rho[alpha] - R[alpha] * omega[alpha]
        dRdt[alpha] -= sum(C[i] * R[alpha] * u[i, alpha] for i in range(N))
        dRdt[alpha] += sum(sum(C[i] * R[beta] * u[i, beta] * l[i, beta, alpha] for beta in range(M)) for i in range(N))
    
    return np.concatenate([dCdt, dRdt])

# Simulation

Once we have the parameters and the derivative equation we are ready to simulate the system and integrate through time. `micrm.py` relies heavily on the brilliant `scipy.integrate` to do the numerical integration and it is worth having a look at the docs to get an idea of what is going on under the hood in your simulations. The simulation procedure is fairly straightforward and just requires that we first create an ODEProblem object which defines the problem for the ODE solver and then solve it with the aptly named solve function. To define the ODEProblem we need to specify the initial state of the system as well as the timespan we want to simulate over:


In [ ]:
from scipy.integrate import solve_ivp

# intial value
C0 = np.full(N,0.01)  # consumer
R0 = np.full(M,1)   # resource
Y0 = np.concatenate([C0, R0])

# time sacle
t_span = (0, 50)
t_eval = np.linspace(*t_span, 300)

# solve ode
sol = solve_ivp(dCdt_Rdt, t_span, Y0, t_eval=t_eval)